In [ ]:
import numpy as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
# import dwf

# from ctypes import *

%load_ext autoreload
%autoreload 2

In [ ]:
from lib import AD2

In [ ]:
interface = AD2.AD2_I2C(voltage=None, pin=0)

In [ ]:
# Check BME680 WHOAMI - should be 97
interface.ReadRegister(0b1110110, 0xD0, bytes_to_read=1)

In [ ]:
led_driver_address = 0b1101000
led_driver_address = 0b1101000
hex(led_driver_address)

# Confusing Definitions

In [ ]:
# PWM - Use Pulse Width Modulation to manually turn LED on and off with certain average power
# One shot mode - Automatic modulation of breathing cycle for LED

## One shot mode settings
# RM - Ramping function, 
# HT - Hold time. When set to 0, LED will be left on. When set to 1, LED will be left off
# BME - Breathing mark enabled? Output pin will notify MCU when to update color data

# Write Register Command

In [ ]:
reg_map = {
    "Shutdown": [0x00, 0, 8, True],
    "EN": [0x00, 5, 1, True],
    "SSD": [0x00, 0, 1, True],
    
    "Breathing Control": [0x01, 0, 8, True],
    "RM": [0x01, 5, 1, True],
    "HT": [0x01, 4, 1, True],
    "BME": [0x01, 2, 1, True],
    "CSS": [0x01, 0, 2, True],
    
    "LED Mode": [0x02, 0, 8, True],
    "RGB": [0x02, 5, 1, True],
    
    "Current Setting": [0x03, 0, 8, True],
    "CS": [0x03, 2, 3, True],
    
    "PWM1": [0x04, 0, 8, True],
    "PWM2": [0x05, 0, 8, True],
    "PWM3": [0x06, 0, 8, True],
    
    "PWM Update": [0x07, 0, 8, True],
    
    "T01": [0x0A, 4, 4, True],
    "T02": [0x0B, 4, 4, True],
    "T03": [0x0C, 4, 4, True],
    
    "T11": [0x10, 5, 3, True],
    "T12": [0x11, 5, 3, True],
    "T13": [0x12, 5, 3, True],
    
    "T21": [0x10, 1, 4, True],
    "T22": [0x11, 1, 4, True],
    "T23": [0x12, 1, 4, True],
    
    "T31": [0x16, 5, 3, True],
    "T32": [0x17, 5, 3, True],
    "T33": [0x18, 5, 3, True],
    
    "T41": [0x16, 1, 4, True],
    "T42": [0x17, 1, 4, True],
    "T43": [0x18, 1, 4, True],
    
    "Time Update": [0x1C, 0, 8, True],
    
    "LED Control": [0x1D, 0, 3, True],
    
    "Reset": [0x2F, 0, 3, True],
}

In [ ]:
current_regs = {}
for register in list(reg_map):
    current_regs[reg_map[register][0]] = 0x00

def write_led(register, field_val):
    if register in list(reg_map):
        reg_address = reg_map[register][0]
        field_offset = reg_map[register][1]
        field_length = reg_map[register][2]
        curr_field_val = current_regs[reg_address]

        # Zero-ing mask
        mask1 = 2 ** field_length - 1
        mask1 = mask1 << field_offset

        mask1 = (2 ** 8 - 1) & ~mask1
        
        # Mask adding actual field_val
        mask2 = field_val << field_offset
        
        # Final masking
        field_val = (curr_field_val & mask1) | mask2

        current_regs[reg_address] = field_val

def write_led_all():
    for key, value in current_regs.items():
        if key == 0x2F:
            continue
        if key == 0x07:
            continue
        if key == 0x1C:
            continue
        # print(hex(key), ', ', '{:08b}'.format(value))
        interface.WriteRegister16(0x68, key, value)

def report(current_regs):
    for key, value in current_regs.items():
        print(hex(key), ', ', '{:08b}'.format(value))

def soft_reset():
    interface.WriteRegister16(0x68, 0x2f, 0)
    for register in list(reg_map):
        current_regs[reg_map[register][0]] = 0x00

def update_chip():
    interface.WriteRegister16(0x68, 0x1C, 0)  # Update time registers
    interface.WriteRegister16(0x68, 0x1D, 0b111)  # Enable LED controls
    # interface.WriteRegister16(0x68, 0x07, 0)  # Load PWM registers and LED control registers DATA

In [ ]:
def set_color(color='red', brightness=50, update=True):

    brightness = int(brightness * 255 / 100)

    if color.upper() == 'RED':
        write_led('PWM1', brightness)  # Write PWM settings for each LED - Red Light
    elif color.upper() == 'GREEN':
        write_led('PWM2', brightness)  # Write PWM settings for each LED - Green Light
    elif color.upper() == 'BLUE':
        write_led('PWM3', brightness)  # Write PWM settings for each LED - Blue Light
    elif color.upper() == 'YELLOW':
        write_led('PWM1', brightness)
        write_led('PWM2', brightness)
    elif color.upper() == 'PURPLE':
        write_led('PWM1', brightness)
        write_led('PWM3', brightness)
    elif color.upper() == 'TEAL':
        write_led('PWM2', brightness)
        write_led('PWM3', brightness)
    elif color.upper() == 'ALL':
        write_led('PWM1', brightness)  # Write PWM settings for each LED - Red Light
        write_led('PWM2', brightness)  # Write PWM settings for each LED - Green Light
        write_led('PWM3', brightness)  # Write PWM settings for each LED - Blue Light
    elif color.upper() == 'WHITE':
        write_led('PWM1', brightness)  # Write PWM settings for each LED - Red Light
        write_led('PWM2', brightness)  # Write PWM settings for each LED - Green Light
        write_led('PWM3', brightness)  # Write PWM settings for each LED - Blue Light

    if update:
        write_led_all()
        update_chip()

# PWM Mode

In [ ]:
soft_reset()
# write_led('Reset', 0b0)  # Reset register map

write_led('EN', 0b1)  # Enable All Channels
write_led('SSD', 0b0)  # Disable software shutdown

write_led('RM', 0b1)  # Enable ramp mode
write_led('HT', 0b0)  # Hold on T2
write_led('BME', 0b1)  # Enable breathing mark
write_led('CSS', 0b01)  # Enable channel 3  - I don't think does anything?

write_led('RGB', 0b0)  # Set to PWM programming mode

write_led('CS', 0b001)  # Set current limits on all LEDs

write_led('LED Control', 0b111)  # Set current limits on all LEDs

write_led_all()

In [ ]:
set_color(color='all', brightness=0)
# set_color(color='YELLOW', brightness=0.5)
# set_color(color='blue', brightness=50)
set_color(color='green', brightness=40)

In [ ]:
set_color(color='all', brightness=0)

In [ ]:
# Sequence for alternating lights
wait_time1 = 0.2
wait_time2 = 0.1
brightness = 20

for _ in range(1):

    # Toggle Red Light on/off
    set_color(color='all', brightness=0)
    time.sleep(wait_time1)
    set_color(color='red', brightness=brightness)
    time.sleep(wait_time2)

    # Toggle Green Light on/off
    set_color(color='all', brightness=0)
    time.sleep(wait_time1)
    set_color(color='green', brightness=brightness)
    time.sleep(wait_time2)

    # Toggle Blue Light on/off
    set_color(color='all', brightness=0)
    time.sleep(wait_time1)
    set_color(color='blue', brightness=brightness)
    time.sleep(wait_time2)

    # Toggle YELLOW Light on/off
    set_color(color='all', brightness=0)
    time.sleep(wait_time1)
    set_color(color='YELLOW', brightness=brightness)
    time.sleep(wait_time2)

    # Toggle PURPLE Light on/off
    set_color(color='all', brightness=0)
    time.sleep(wait_time1)
    set_color(color='PURPLE', brightness=brightness)
    time.sleep(wait_time2)

    # Toggle TEAL Light on/off
    set_color(color='all', brightness=0)
    time.sleep(wait_time1)
    set_color(color='TEAL', brightness=brightness)
    time.sleep(wait_time2)

    # Toggle WHITE Light on/off
    set_color(color='all', brightness=0)
    time.sleep(wait_time1)
    set_color(color='WHITE', brightness=brightness)
    time.sleep(wait_time2)

set_color(color='all', brightness=0)

In [ ]:
set_color(color='all', brightness=0)
# set_color(color='teal', brightness=10)

# One Shot Mode

In [ ]:
soft_reset()
# write_led('Reset', 0b0)  # Reset register map

write_led('EN', 0b1)  # Enable All Channels
write_led('SSD', 0b0)  # Disable software shutdown

write_led('RM', 0b0)  # Enable ramp mode
write_led('HT', 0b1)  # Hold on T2
write_led('BME', 0b1)  # Enable breathing mark
write_led('CSS', 0b10)  # Enable channel 3  - Not sure what this is doing?

write_led('RGB', 0b1)  # Set to one shot programming mode

# write_led('CS', 0b001)  # Set current limits on all LEDs - 10mA
write_led('CS', 0b000)  # Set current limits on all LEDs - 42mA

write_led('LED Control', 0b111)  # Set current limits on all LEDs

# set_color(color='green', brightness=50, update=False)
# set_color(color='blue', brightness=10, update=False)

# set_color(color='red', brightness=80, update=False)
# set_color(color='green', brightness=20, update=False)

In [ ]:
# Battery Low
set_color(color='white', brightness=0, update=False)
set_color(color='red', brightness=5, update=False)

t0 = 0b0000
t1 = 0b000
t2 = 0b0000
t3 = t1
t4 = 0b0100

write_led('T01', t0)  # Set T0 in shot mode
write_led('T02', t0)  # Set T0 in shot mode
write_led('T03', t0)  # Set T0 in shot mode

write_led('T11', t1)  # Set T1 in shot mode
write_led('T12', t1)  # Set T1 in shot mode
write_led('T13', t1)  # Set T1 in shot mode

write_led('T21', t2)  # Set T2 in shot mode
write_led('T22', t2)  # Set T2 in shot mode
write_led('T23', t2)  # Set T2 in shot mode

write_led('T31', t3)  # Set T3 in shot mode
write_led('T32', t3)  # Set T3 in shot mode
write_led('T33', t3)  # Set T3 in shot mode

write_led('T41', t4)  # Set T4 in shot mode
write_led('T42', t4)  # Set T4 in shot mode
write_led('T43', t4)  # Set T4 in shot mode

write_led_all()

update_chip()

In [ ]:
# Charging (Low battery)
set_color(color='white', brightness=0, update=False)
set_color(color='green', brightness=100, update=False)

t0 = 0b0000
t1 = 0b001
t2 = 0b0000
t3 = 0b001
t4 = 0b0000

write_led('T01', t0)  # Set T0 in shot mode
write_led('T02', t0)  # Set T0 in shot mode
write_led('T03', t0)  # Set T0 in shot mode

write_led('T11', t1)  # Set T1 in shot mode
write_led('T12', t1)  # Set T1 in shot mode
write_led('T13', t1)  # Set T1 in shot mode

write_led('T21', t2)  # Set T2 in shot mode
write_led('T22', t2)  # Set T2 in shot mode
write_led('T23', t2)  # Set T2 in shot mode

write_led('T31', t3)  # Set T3 in shot mode
write_led('T32', t3)  # Set T3 in shot mode
write_led('T33', t3)  # Set T3 in shot mode

write_led('T41', t4)  # Set T4 in shot mode
write_led('T42', t4)  # Set T4 in shot mode
write_led('T43', t4)  # Set T4 in shot mode

write_led_all()

update_chip()

In [ ]:
# Charging (Med battery)
set_color(color='white', brightness=0, update=False)
set_color(color='green', brightness=100, update=False)

t0 = 0b0000
t1 = 0b001
t2 = 0b0001
t3 = 0b001
t4 = 0b0010

write_led('T01', t0)  # Set T0 in shot mode
write_led('T02', t0)  # Set T0 in shot mode
write_led('T03', t0)  # Set T0 in shot mode

write_led('T11', t1)  # Set T1 in shot mode
write_led('T12', t1)  # Set T1 in shot mode
write_led('T13', t1)  # Set T1 in shot mode

write_led('T21', t2)  # Set T2 in shot mode
write_led('T22', t2)  # Set T2 in shot mode
write_led('T23', t2)  # Set T2 in shot mode

write_led('T31', t3)  # Set T3 in shot mode
write_led('T32', t3)  # Set T3 in shot mode
write_led('T33', t3)  # Set T3 in shot mode

write_led('T41', t4)  # Set T4 in shot mode
write_led('T42', t4)  # Set T4 in shot mode
write_led('T43', t4)  # Set T4 in shot mode

write_led_all()

update_chip()

In [ ]:
# Charging (high battery)
set_color(color='white', brightness=0, update=False)
set_color(color='green', brightness=100, update=False)

t0 = 0b0000
t1 = 0b010
t2 = 0b0100
t3 = 0b010
t4 = 0b0001

write_led('T01', t0)  # Set T0 in shot mode
write_led('T02', t0)  # Set T0 in shot mode
write_led('T03', t0)  # Set T0 in shot mode

write_led('T11', t1)  # Set T1 in shot mode
write_led('T12', t1)  # Set T1 in shot mode
write_led('T13', t1)  # Set T1 in shot mode

write_led('T21', t2)  # Set T2 in shot mode
write_led('T22', t2)  # Set T2 in shot mode
write_led('T23', t2)  # Set T2 in shot mode

write_led('T31', t3)  # Set T3 in shot mode
write_led('T32', t3)  # Set T3 in shot mode
write_led('T33', t3)  # Set T3 in shot mode

write_led('T41', t4)  # Set T4 in shot mode
write_led('T42', t4)  # Set T4 in shot mode
write_led('T43', t4)  # Set T4 in shot mode

write_led_all()

update_chip()

In [ ]:
# Full battery, finished charging
set_color(color='white', brightness=0, update=False)
set_color(color='green', brightness=40, update=False)

t0 = 0b0000
t1 = 0b0000
t2 = 0b1000
t3 = 0b0000
t4 = 0b0000

write_led('T01', t0)  # Set T0 in shot mode
write_led('T02', t0)  # Set T0 in shot mode
write_led('T03', t0)  # Set T0 in shot mode

write_led('T11', t1)  # Set T1 in shot mode
write_led('T12', t1)  # Set T1 in shot mode
write_led('T13', t1)  # Set T1 in shot mode

write_led('T21', t2)  # Set T2 in shot mode
write_led('T22', t2)  # Set T2 in shot mode
write_led('T23', t2)  # Set T2 in shot mode

write_led('T31', t3)  # Set T3 in shot mode
write_led('T32', t3)  # Set T3 in shot mode
write_led('T33', t3)  # Set T3 in shot mode

write_led('T41', t4)  # Set T4 in shot mode
write_led('T42', t4)  # Set T4 in shot mode
write_led('T43', t4)  # Set T4 in shot mode

write_led_all()

update_chip()

In [ ]:
# Air Quality Bad
set_color(color='white', brightness=0, update=False)
set_color(color='yellow', brightness=5, update=False)

t0 = 0b0000
t1 = 0b001
t2 = 0b0000
t3 = t1
t4 = 0b0100

write_led('T01', t0)  # Set T0 in shot mode
write_led('T02', t0)  # Set T0 in shot mode
write_led('T03', t0)  # Set T0 in shot mode

write_led('T11', t1)  # Set T1 in shot mode
write_led('T12', t1)  # Set T1 in shot mode
write_led('T13', t1)  # Set T1 in shot mode

write_led('T21', t2)  # Set T2 in shot mode
write_led('T22', t2)  # Set T2 in shot mode
write_led('T23', t2)  # Set T2 in shot mode

write_led('T31', t3)  # Set T3 in shot mode
write_led('T32', t3)  # Set T3 in shot mode
write_led('T33', t3)  # Set T3 in shot mode

write_led('T41', t4)  # Set T4 in shot mode
write_led('T42', t4)  # Set T4 in shot mode
write_led('T43', t4)  # Set T4 in shot mode

write_led_all()

update_chip()

In [ ]:
# Air Quality Really Bad
set_color(color='white', brightness=0, update=False)
set_color(color='yellow', brightness=5, update=False)

t0 = 0b0000
t1 = 0b001
t2 = 0b0000
t3 = t1
t4 = 0b0100

write_led('T01', t0)  # Set T0 in shot mode
write_led('T02', t0)  # Set T0 in shot mode
write_led('T03', t0)  # Set T0 in shot mode

write_led('T11', t1)  # Set T1 in shot mode
write_led('T12', t1)  # Set T1 in shot mode
write_led('T13', t1)  # Set T1 in shot mode

write_led('T21', t2)  # Set T2 in shot mode
write_led('T22', t2)  # Set T2 in shot mode
write_led('T23', t2)  # Set T2 in shot mode

write_led('T31', t3)  # Set T3 in shot mode
write_led('T32', t3)  # Set T3 in shot mode
write_led('T33', t3)  # Set T3 in shot mode

write_led('T41', t4)  # Set T4 in shot mode
write_led('T42', t4)  # Set T4 in shot mode
write_led('T43', t4)  # Set T4 in shot mode

write_led_all()

update_chip()